In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# 한글 설정
import matplotlib as mpl
mpl.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# import matplotlib.font_manager as fm
# font_name = fm.FontProperties(fname="/Users/dofany/dofany_study/python/analysis/data/ImcreSoojin OTF.otf").get_name()
# plt.rc("font", family=font_name)

import seaborn as sns

plt.style.use("fivethirtyeight")

import warnings
warnings.filterwarnings("ignore")

import requests
from urllib.request import urlopen 
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from selenium import webdriver

# 셀프 주유소는 과연 저렴한가?
+ 실제 셀프 주유소가 저렴한지에 대한 팩트 체크
+ 가장 싼 주유소는 어디인가?
+ http://www.opinet.co.kr

## 서울시 구별 주유소 가격 정보 가져오기

### Sample Code

#### 원격 제어

In [74]:
driver = webdriver.Chrome("/Users/dofany/Downloads/chromedriver")
driver.get("https://www.opinet.co.kr/searRgSelect.do")

In [10]:
driver.close()

#### 시 목록 가져오기

In [18]:
si_list_raw = driver.find_element_by_xpath("""//*[@id="SIDO_NM0"]""")
si_list = si_list_raw.find_elements_by_tag_name("option")
si_list
si_list[1].get_attribute('value')

si_names = [option.get_attribute('value') for option in si_list]
si_names.remove("")
si_names

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

#### 시 선택하기

In [26]:
element = driver.find_element_by_id("SIDO_NM0")
element.send_keys(si_names[0])

#### 구 목록 가져오기

In [36]:
gu_list_raw = driver.find_element_by_xpath("""//*[@id="SIGUNGU_NM0"]""")
gu_list = gu_list_raw.find_elements_by_tag_name("option")
gu_list
gu_list[1].get_attribute('value')

gu_names = [option.get_attribute('value') for option in gu_list]
gu_names.remove("")
gu_names

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

#### 구 선택하기 

In [49]:
element = driver.find_element_by_id("SIGUNGU_NM0")
element.send_keys(gu_names[17])

#### 조회 버튼 누르기

In [52]:
search = """//*[@id="searRgSelect"]"""
driver.find_element_by_xpath(search).click()

#### 엑셀 저장버튼 누르기

In [53]:
search = """//*[@id="glopopd_excel"]"""
driver.find_element_by_xpath(search).click()

## 지역별(구 별)로 주유소 정보를 엑셀파일로 저장

In [72]:
from tqdm import tqdm_notebook
import time

In [76]:
for gu in tqdm_notebook(gu_names):
    element = driver.find_element_by_id("SIGUNGU_NM0")
    element.send_keys(gu)
    
    time.sleep(1)
    
    search = """//*[@id="searRgSelect"]"""
    driver.find_element_by_xpath(search).click()
    
    time.sleep(1)
    
    search = """//*[@id="glopopd_excel"]"""
    driver.find_element_by_xpath(search).click()
    
driver.close()

  0%|          | 0/25 [00:00<?, ?it/s]

# Data Preprocessing

## 저장된 엑셀 파일 불러오기

In [89]:
from glob import glob

In [95]:
station_files = glob("data/oil/지역*.*")
station_files

tmp_raw = []
for fname in station_files:
    tmp = pd.read_excel(fname, header = 2)
    tmp_raw.append(tmp)

In [96]:
len(tmp_raw)

25

### 데이터 프레임으로 합치기

In [130]:
station_raw = pd.concat(tmp_raw)
station_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 0 to 9
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      486 non-null    object
 1   상호      486 non-null    object
 2   주소      486 non-null    object
 3   상표      486 non-null    object
 4   전화번호    486 non-null    object
 5   셀프여부    486 non-null    object
 6   고급휘발유   486 non-null    object
 7   휘발유     486 non-null    object
 8   경유      486 non-null    object
 9   실내등유    486 non-null    object
dtypes: object(10)
memory usage: 41.8+ KB


In [131]:
# 재구성
stations = pd.DataFrame({'Oil_store':station_raw['상호'],
              "주소":station_raw["주소"],
              "가격":station_raw["휘발유"],
              "셀프":station_raw["셀프여부"],
              "상표":station_raw["상표"]})
stations.head()

,Oil_store,주소,가격,셀프,상표
0,도봉제일주유소(주)송만에너지,서울 도봉구 도봉로 783 (도봉동),1529,N,현대오일뱅크
1,한이에너지(주) 쌍문주유소,서울 도봉구 도봉로 547 (쌍문동),1534,Y,S-OIL
2,정다운셀프주유소,서울 도봉구 도봉로 635,1534,Y,SK에너지
3,구도일주유소 파크빌,서울 도봉구 해등로3길 86 (창동),1534,Y,S-OIL
4,구도일주유소 도봉,서울 도봉구 도봉로 706 (방학동),1534,Y,S-OIL


### 구로 나누기

In [132]:
stations['구'] = [addr.split()[1] for addr in stations['주소']]
stations.head()

,Oil_store,주소,가격,셀프,상표,구
0,도봉제일주유소(주)송만에너지,서울 도봉구 도봉로 783 (도봉동),1529,N,현대오일뱅크,도봉구
1,한이에너지(주) 쌍문주유소,서울 도봉구 도봉로 547 (쌍문동),1534,Y,S-OIL,도봉구
2,정다운셀프주유소,서울 도봉구 도봉로 635,1534,Y,SK에너지,도봉구
3,구도일주유소 파크빌,서울 도봉구 해등로3길 86 (창동),1534,Y,S-OIL,도봉구
4,구도일주유소 도봉,서울 도봉구 도봉로 706 (방학동),1534,Y,S-OIL,도봉구


In [133]:
print(stations["구"].unique())
print(len(stations["구"].unique()))

['도봉구' '마포구' '광진구' '중랑구' '관악구' '서대문구' '노원구' '강서구' '양천구' '종로구' '서초구' '성동구'
 '중구' '영등포구' '강북구' '용산구' '강동구' '성북구' '송파구' '은평구' '동대문구' '강남구' '금천구' '구로구'
 '동작구']
25


### 가격이 없는 데이터 제거

In [134]:
stations['가격'].unique()
stations[stations['가격']=="-"]
stations = stations[stations['가격']!="-"]

stations["가격"] = [float(val) for val in stations['가격']]

stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474 entries, 0 to 9
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Oil_store  474 non-null    object 
 1   주소         474 non-null    object 
 2   가격         474 non-null    float64
 3   셀프         474 non-null    object 
 4   상표         474 non-null    object 
 5   구          474 non-null    object 
dtypes: float64(1), object(5)
memory usage: 25.9+ KB


### 인덱스 조정

In [135]:
stations.reset_index(inplace=True)
stations.head()

,index,Oil_store,주소,가격,셀프,상표,구
0,0,도봉제일주유소(주)송만에너지,서울 도봉구 도봉로 783 (도봉동),1529.0,N,현대오일뱅크,도봉구
1,1,한이에너지(주) 쌍문주유소,서울 도봉구 도봉로 547 (쌍문동),1534.0,Y,S-OIL,도봉구
2,2,정다운셀프주유소,서울 도봉구 도봉로 635,1534.0,Y,SK에너지,도봉구
3,3,구도일주유소 파크빌,서울 도봉구 해등로3길 86 (창동),1534.0,Y,S-OIL,도봉구
4,4,구도일주유소 도봉,서울 도봉구 도봉로 706 (방학동),1534.0,Y,S-OIL,도봉구


In [136]:
del stations['index']

In [137]:
stations

,Oil_store,주소,가격,셀프,상표,구
0,도봉제일주유소(주)송만에너지,서울 도봉구 도봉로 783 (도봉동),1529.0,N,현대오일뱅크,도봉구
1,한이에너지(주) 쌍문주유소,서울 도봉구 도봉로 547 (쌍문동),1534.0,Y,S-OIL,도봉구
2,정다운셀프주유소,서울 도봉구 도봉로 635,1534.0,Y,SK에너지,도봉구
3,구도일주유소 파크빌,서울 도봉구 해등로3길 86 (창동),1534.0,Y,S-OIL,도봉구
4,구도일주유소 도봉,서울 도봉구 도봉로 706 (방학동),1534.0,Y,S-OIL,도봉구
...,...,...,...,...,...,...
469,현대오일뱅크㈜직영 흑석동셀프주유소,서울 동작구 현충로 101 (흑석동),1608.0,Y,현대오일뱅크,동작구
470,현대오일뱅크(주)직영 동작주유소,서울 동작구 상도로 403 (상도동),1614.0,N,현대오일뱅크,동작구
471,대성산업(주) 노량진주유소,서울 동작구 노량진로 172,1628.0,N,GS칼텍스,동작구
472,(주)성림에너지 남성주유소,서울 동작구 동작대로 135 (사당동),1629.0,Y,GS칼텍스,동작구


## EDA